In [1]:
# Ok so first of all the instructions here (https://cryptopals.com/sets/5/challenges/39) leave a lot out
# And the exercise doesn't work if you follow the instructions specifically... they leave out a lot about the totient function
# I followed Wikipedia instead (https://en.m.wikipedia.org/wiki/RSA_(cryptosystem))


from Crypto.Util.number import getPrime
Prime = lambda : getPrime(1024)

In [2]:
# Using implementations from https://github.com/ricpacca/cryptopals/blob/master/S5C39.py

def GCD(a, b):
    """Computes the greatest common divisor between a and b using the Euclidean algorithm."""
    while b != 0:
        a, b = b, a % b

    return a


def LCM(a, b):
    """Computes the lowest common multiple between a and b using the GCD method."""
    return a // GCD(a, b) * b


def ModInv(a, n):
    """Computes the multiplicative inverse of a modulo n using the extended Euclidean algorithm."""
    t, r = 0, n
    new_t, new_r = 1, a

    while new_r != 0:
        quotient = r // new_r
        t, new_t = new_t, t - quotient * new_t
        r, new_r = new_r, r - quotient * new_r

    if r > 1:
        raise Exception("a is not invertible")
    if t < 0:
        t = t + n

    return t

assert ModInv(17, 3120) == 2753

In [3]:
# Let e be 3
e = 3

In [4]:
# The requirement regarding GCD is specified in Wikipedia, NOT the challenge itself

def GeneratePrimes(e = e):
    while True:
        p, q = Prime(), Prime()
        n = p*q
        et = LCM(p-1, q-1) % n

        if 2 < e < et and GCD(e, et) == 1:
            return p, q, et

p, q, et = GeneratePrimes()

In [5]:
d = ModInv(e, et)
print(d)

assert (d*e) % et == 1

2531945975104390738956315826950012248627491422779152544185467871658026598682687417450223536252231575353806644229988447794000897862175963216933862756850912565585221417101525688293511192390426963547522216094567663393993445480560072864730837410109696273190184472872422498714973917733467239338975975079262317330118263034615855723983942727849350675430250497782945719071453853607083842979140324451854760406951223607218055910361253139865867892325539771199645554588388857488144459317771938249569384258811681804072650254876234669593798149223382749023275810594251057153849589127379618925874481862843016055746319368029055175117


In [6]:
class RSAServer(object):
    def __init__(self, e = 3):
        self.e = e
        p, q, et = GeneratePrimes(e)
        self.p = p
        self.q = q
        self.d = ModInv(e, et)
        self.n = p*q
        
    def GetPubkey(self):
        return self.e, self.n
    
    def Decrypt(self, c):
        m = pow(c, self.d, self.n)
        return m.to_bytes(byteorder='big', length = 2*1024//8)

class RSAClient(object):
    def __init__(self, e, n):
        self.e = e
        self.n = n
        
    def Encrypt(self, message):
        m = message if type(message) is int else int.from_bytes(message, byteorder = 'big')
        assert m < self.n
        return pow(m, self.e, self.n)
        

In [7]:
# Make three client-server pairs with three different pubkeys

server0 = RSAServer()
server1 = RSAServer()
server2 = RSAServer()

pubkey0 = server0.GetPubkey()
pubkey1 = server1.GetPubkey()
pubkey2 = server2.GetPubkey()

assert len({pubkey0, pubkey1, pubkey2}) == 3 # Make sure all pubkeys are different

client0 = RSAClient(*pubkey0)
client1 = RSAClient(*pubkey1)
client2 = RSAClient(*pubkey2)

n_values = tuple([ n for (e, n) in [pubkey0, pubkey1, pubkey2] ])

In [8]:
message = b"""
But, soft! what light through yonder window breaks? 
It is the east, and Juliet is the sun.  
Arise, fair sun, and kill the envious moon,
Who is already sick and pale with grief,
That thou her maid art far more fair than she.
"""


ciphertext0 = client0.Encrypt(message)
ciphertext1 = client1.Encrypt(message)
ciphertext2 = client2.Encrypt(message)

In [9]:
# Make sure only the correct server can encrypte each ciphertext

assert server0.Decrypt(ciphertext0).replace(b'\0', b'') == message
assert server1.Decrypt(ciphertext0).replace(b'\0', b'') != message
assert server2.Decrypt(ciphertext0).replace(b'\0', b'') != message

In [10]:
# Now decrypt

m_s_0 = n_values[1] * n_values[2]
m_s_1 = n_values[2] * n_values[0]
m_s_2 = n_values[0] * n_values[1]

N_012 = n_values[0] * n_values[1] * n_values[2]

result = ciphertext0 * m_s_0 * ModInv(m_s_0, n_values[0]) + \
         ciphertext1 * m_s_1 * ModInv(m_s_1, n_values[1]) + \
         ciphertext2 * m_s_2 * ModInv(m_s_2, n_values[2])
result %= N_012

In [25]:
def CubeRoot(n):
    high = n
    low = 0
    guess = None
    
    while True:
        last_guess = guess
        guess = (high+low)//2
        product = guess*guess*guess
        if product == n:
            return guess
        elif last_guess == guess:
            return None
        elif product > n:
            high = guess
        elif product < n:
            low = guess+1
        else:
            return None

In [31]:
root = CubeRoot(64)
root

4

In [35]:
root = CubeRoot(result)
decrypted = root.to_bytes(byteorder='big', length = 2*1024//8).replace(b'\0', b'')
assert decrypted == message
print(decrypted)

b'\nBut, soft! what light through yonder window breaks? \nIt is the east, and Juliet is the sun.  \nArise, fair sun, and kill the envious moon,\nWho is already sick and pale with grief,\nThat thou her maid art far more fair than she.\n'


In [36]:
!pwd

1707.38s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


/Users/bloomfield/git/cryptopals/cryptopals/set5/set5challenge40
